In [68]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 

from transformers import BertForSequenceClassification, BertTokenizer, BertForMaskedLM

from simpletransformers.language_modeling import LanguageModelingModel

from sklearn.metrics.pairwise import cosine_similarity, paired_euclidean_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import normalize, StandardScaler, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import itertools
from collections import Counter

from tqdm import tqdm
import torch

import networkx as nx

import matplotlib.pyplot as plt

import plotly.graph_objects as go
from functools import partial

import pickle

from collections import deque

from torchvision import models
import torch.nn.functional as F
from torch import nn

from sklearn.utils import resample

import fasttext
import sister



from typing import List
from pathlib import Path
from joblib import dump, load

import plotly.graph_objects as go

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/home/roshansk/YelpAnalysis/')
from utils import *

import spacy

# ft = fasttext.load_model('/home/roshansk/YelpAnalysis/RandomForest/cc.en.300.bin')

# embedder = sister.MeanEmbedding(lang="en")


from sklearn.svm import LinearSVC


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
nlp = spacy.load('/data2/link10/models/fasttext/en_fasttext_crawl')
# nlp = spacy.load('/data2/link10/models/fasttext/en_fasttext_crawl_subword/')

sys.path.insert(0,'/home/roshansk/YelpAnalysis/Datasets/')
from createDataset import *

dataFolder = '/home/roshansk/YelpAnalysis/Datasets/'

### Helper Functions

In [47]:
def trainSVM(trainData, testData, trainDf, testDf):
  
    model = LinearSVC(random_state=1, dual=False, max_iter=10000) 
    model.fit(trainData, trainDf.label)  
    pred = model.predict(testData)
         
    return model
    
    
def generateLexicon(NNnet, trainDf, nlp):
    return getWordPred_SVM(model, trainDf, nlp)

def testSVM_direct( model, dataset, lexiconWords, lexiconMap, nlp,  train = False):
    
    if dataset != 'dialog':
        trainDf, devDf, testDf = getData(dataFolder, dataset)
    else:
        testDf = getExternalData(dataFolder, 'dialog')

        
    if train:
        testData = generateFastTextData_Spacy(trainDf, nlp, textVariable = 'text')
        testDf = trainDf
    else:
        testData = generateFastTextData_Spacy(testDf, nlp, textVariable = 'text')
        
    
    pred = model.predict(testData)
    
    modelAcc = np.round(accuracy_score(testDf.label, pred),3)
    modelF1 = np.round(f1_score(testDf.label, pred),3)
    
#     print('Model Evaluation')
#     print(f"Acc : {acc} F1 : {f1}" )
    
#     print("Lexicon Evalution")
    lexAcc, lexF1 = evaluateLexicon(testDf, lexiconWords, lexiconMap, nlp, returnScores = True)
    
    print(f"{dataset} , {modelAcc} , {modelF1} , {lexAcc} , {lexF1}")
    

### Load Data & Model Training

In [77]:
trainDf, devDf, testDf = getData(dataFolder, 'nrc_joy')

In [78]:
trainData = generateFastTextData_Spacy(trainDf, nlp, textVariable = 'text')

testData = generateFastTextData_Spacy(testDf, nlp, textVariable = 'text')



In [79]:
model = trainSVM(trainData, testData, trainDf, testDf)

In [80]:
lexicon = generateLexicon(model,trainDf, nlp)

In [81]:
lexicon.head()

,word,score,wordCount,SVM_Rank
7478,joy-,13.591259,2,0
18074,lemsip,11.103914,2,1
6969,carols,10.273476,13,2
24256,unltd,9.931378,1,3
10966,m25,9.796602,3,4


In [82]:
lexiconWords, lexiconMap = getLexicon(0, df = lexicon)

### Experiments

In [48]:
testSVM_direct(model, 'dialog',lexiconWords, lexiconMap, nlp)

100%|██████████| 13538/13538 [00:05<00:00, 2636.24it/s]


dialog , 0.739 , 0.82 , 0.838 , 0.907


In [49]:
testSVM_direct(model, 'amazon_finefood_subset',lexiconWords, lexiconMap, nlp)

100%|██████████| 1000/1000 [00:00<00:00, 1242.57it/s]


amazon_finefood_subset , 0.756 , 0.839 , 0.874 , 0.929


In [50]:
testSVM_direct(model, 'yelp_subset',lexiconWords, lexiconMap, nlp)

100%|██████████| 1000/1000 [00:00<00:00, 1189.93it/s]


yelp_subset , 0.879 , 0.917 , 0.888 , 0.927


In [51]:
testSVM_direct(model, 'yelp_subset',lexiconWords, lexiconMap, nlp, train = True)

100%|██████████| 12452/12452 [00:10<00:00, 1189.99it/s]


yelp_subset , 0.908 , 0.907 , 0.909 , 0.908


In [52]:
testSVM_direct(model, 'nrc_joy',lexiconWords, lexiconMap, nlp)

100%|██████████| 2105/2105 [00:00<00:00, 2468.06it/s]


nrc_joy , 0.589 , 0.531 , 0.628 , 0.369


In [53]:
testSVM_direct(model, 'empathy',lexiconWords, lexiconMap, nlp)

100%|██████████| 186/186 [00:00<00:00, 1410.98it/s]


empathy , 0.516 , 0.118 , 0.532 , 0.592


In [54]:
testSVM_direct(model, 'amazon_toys_subset',lexiconWords, lexiconMap, nlp)

100%|██████████| 1000/1000 [00:01<00:00, 958.50it/s]


amazon_toys_subset , 0.867 , 0.926 , 0.935 , 0.966


In [56]:
testSVM_direct(model, 'yelp_1v5',lexiconWords, lexiconMap, nlp)

100%|██████████| 60579/60579 [00:53<00:00, 1135.33it/s]


yelp_1v5 , 0.952 , 0.967 , 0.958 , 0.972


In [83]:
dataList = ['dialog','amazon_finefood_subset','amazon_toys_subset','yelp_subset','empathy','nrc_joy']
for data in dataList:
    testSVM_direct(model, data,lexiconWords, lexiconMap, nlp)
    

dialog , 0.556 , 0.647 , 0.819 , 0.899
amazon_finefood_subset , 0.398 , 0.468 , 0.851 , 0.919
amazon_toys_subset , 0.354 , 0.48 , 0.94 , 0.969
yelp_subset , 0.497 , 0.52 , 0.778 , 0.869
empathy , 0.484 , 0.02 , 0.532 , 0.603
nrc_joy , 0.747 , 0.704 , 0.752 , 0.676
